## Dataset class

In [3]:
import torch
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

import os
import pandas as pd
from torchvision.io import read_image

class ChineseMINISTDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_tranform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx,1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

## Data preprocess

In [4]:
from torchvision.transforms import ToTensor, Lambda

transform = Lambda(lambda y : y.float())
# transform the integer label to one-hot encoding vector
target_transform = Lambda(lambda y : torch.zeros(10, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y-1), value=1))

## NN Class

In [6]:
from torch import nn
from torch.utils.data import DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=16,            
                kernel_size=5,              
                stride=1,                   
                padding=2,                  
            ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(16, 32, 5, 1, 2),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        # fully connected layer, output 10 classes
        self.out = nn.Linear(8192, 10)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
       
        x = x.view(x.size(0), -1)       
        output = self.out(x)
        return output 

Using cpu device


## Load Model

In [7]:
weight_path = 'model_weights.pth'
model = CNN()
model.load_state_dict(torch.load(weight_path))
model.eval()

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=8192, out_features=10, bias=True)
)

## Preapare Testing Data

In [11]:
img_dir = 'numbers'
testing_path = 'testing_set.csv'
testing_datasets = ChineseMINISTDataset(testing_path, img_dir, transform, target_transform)
test_dataloader = DataLoader(testing_datasets, batch_size=32)
loss_fn = loss_fn = nn.CrossEntropyLoss()
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [12]:
test_loop(test_dataloader, model, loss_fn)

Test Error: 
 Accuracy: 92.5%, Avg loss: 0.383298 



## Done